## Lab2 

ANLY-580 Peijin Li

#### Part I: Introduction to Spacy 

Download Spacy's base English language pipeline components

In [1]:
import spacy

In [43]:
pipeline = spacy.load('en_core_web_sm')

Download the 20 News Groups dataset using the sklearn package

In [44]:
from sklearn.datasets import fetch_20newsgroups
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

ng_train = fetch_20newsgroups(subset='train')
ng_test = fetch_20newsgroups(subset='test')
target_names = ng_train.target_names
target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Get the number of training & test examples

In [4]:
len(ng_train.data), len(ng_test.data)

(11314, 7532)

Take a peek at the first document and its label

In [5]:
ng_train.data[0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

In [6]:
label_idx = ng_train.target[0]
ng_train.target_names[label_idx]

'rec.autos'

Evaluate Spacy's recognition of entities, POS

In [45]:
from pprint import pprint

doc = pipeline(ng_train.data[0])
for i, token in enumerate(doc):
    pprint({"text": token.text,
            "lemma": token.lemma_,
            "POS": token.pos_,
            "tag": token.tag_,
            "dep": token.dep_,
            "shape": token.shape_,
            "is_alpha": token.is_alpha,
            "is_stop": token.is_stop})
    if i == 3:
        break

{'POS': 'ADP',
 'dep': 'prep',
 'is_alpha': True,
 'is_stop': True,
 'lemma': 'from',
 'shape': 'Xxxx',
 'tag': 'IN',
 'text': 'From'}
{'POS': 'PUNCT',
 'dep': 'punct',
 'is_alpha': False,
 'is_stop': False,
 'lemma': ':',
 'shape': ':',
 'tag': ':',
 'text': ':'}
{'POS': 'PROPN',
 'dep': 'pobj',
 'is_alpha': False,
 'is_stop': False,
 'lemma': 'lerxst@wam.umd.edu',
 'shape': 'xxxx@xxx.xxx.xxx',
 'tag': 'NNP',
 'text': 'lerxst@wam.umd.edu'}
{'POS': 'PUNCT',
 'dep': 'punct',
 'is_alpha': False,
 'is_stop': False,
 'lemma': '(',
 'shape': '(',
 'tag': '-LRB-',
 'text': '('}


Visualize Spacy's dependency parse

In [46]:
from spacy import displacy

displacy.render(doc, style='dep')

Let's define a preprocessing function that cleans our data

In [47]:
import re
from spacy.language import Language


# http://emailregex.com/
email_re = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""

# replace = [ (pattern-to-replace, replacement),  ...]
replace = [
    (r"<a[^>]*>(.*?)</a>", r"\1"),  # Matches most URLs
    (email_re, "email"),            # Matches emails
    (r"(?<=\d),(?=\d)", ""),        # Remove commas in numbers
    (r"\d+", "numbr"),              # Map digits to special token <numbr>
    (r"[\t\n\r\*\.\@\,\-\/]", " "),   # Punctuation and other junk
    (r"\s+", " ")                   # Stips extra whitespace
]

train_text = ng_train.data
test_text = ng_test.data
for repl in replace:
    train_text = [re.sub(repl[0], repl[1], text) for text in train_text]
    test_text = [re.sub(repl[0], repl[1], text) for text in test_text]

@Language.component("ng20")
def ng20_preprocess(doc):
    tokens = [token for token in doc 
              if not any((token.is_stop, token.is_punct))]
    tokens = [token.lemma_.lower().strip() for token in tokens]
    tokens = [token for token in tokens if token]
    return " ".join(tokens)

pipeline.add_pipe("ng20");

In [10]:
test_text

['From: email (NEIL B GANDLER) Subject: Need info on numbr numbr Bonneville Organization: University at Buffalo Lines: numbr News Software: VAX VMS VNEWS numbr numbr Nntp Posting Host: ubvmsd cc buffalo edu I am a little confused on all of the models of the numbr numbr bonnevilles I have heard of the LE SE LSE SSE SSEI Could someone tell me the differences are far as features or performance I am also curious to know what the book value is for prefereably the numbr model And how much less than book value can you usually get them for In other words how much are they in demand this time of year I have heard that the mid spring early summer is the best time to buy Neil Gandler ',
 'From: Rick Miller <email> Subject: X Face? Organization: Just me Lines: numbr Distribution: world NNTP Posting Host: numbr numbr numbr numbr Summary: Go ahead swamp me <EEP!> I\'m not familiar at all with the format of these "X Face:" thingies but after seeing them in some folks\' headers I\'ve got to see them (

Peek at our processing pipeline

In [48]:
pipeline.analyze_pipes(pretty=True)


============================= Pipeline Overview =============================

#   Component         Assigns               Requires   Scores             Retokenizes
-   ---------------   -------------------   --------   ----------------   -----------
0   tok2vec           doc.tensor                                          False      
                                                                                     
1   tagger            token.tag                        tag_acc            False      
                                                                                     
2   parser            token.dep                        dep_uas            False      
                      token.head                       dep_las                       
                      token.is_sent_start              dep_las_per_type              
                      doc.sents                        sents_p                       
                                                       sents

{'summary': {'tok2vec': {'assigns': ['doc.tensor'],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'tagger': {'assigns': ['token.tag'],
   'requires': [],
   'scores': ['tag_acc'],
   'retokenizes': False},
  'parser': {'assigns': ['token.dep',
    'token.head',
    'token.is_sent_start',
    'doc.sents'],
   'requires': [],
   'scores': ['dep_uas',
    'dep_las',
    'dep_las_per_type',
    'sents_p',
    'sents_r',
    'sents_f'],
   'retokenizes': False},
  'attribute_ruler': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'lemmatizer': {'assigns': ['token.lemma'],
   'requires': [],
   'scores': ['lemma_acc'],
   'retokenizes': False},
  'ner': {'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'],
   'requires': [],
   'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'],
   'retokenizes': False},
  'ng20': {'assigns': [], 'requires': [], 'scores': [], 'retokenizes': False}},
 'problems': {'tok2vec': [],
  'tagger': [],
 

Now pass each training and test document through the pipeline

In [12]:
docs_train = [pipeline(doc) for doc in train_text[:500]]
docs_test = [pipeline(doc) for doc in test_text[:500]]

Let's look at that first document following this transformation and compare it to the original text

In [13]:
ng_train.data[0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

In [14]:
docs_train[0]

'email thing subject car nntp posting host racnumbr wam umd edu organization university maryland college park lines numbr wonder enlighten car see day numbr door sport car look late numbrs early numbrs call bricklin door small addition bumper separate rest body know tellme model engine spec year production car history info funky look car e mail thanks il bring neighborhood lerxst'

#### Part II: Lexical diversity 

##### Task I
Sometimes it's useful to understand how diverse is the language in some body of text. Once simple heuristic to evaluate diversity is as follows: 

$$ lexical\_diversity = \frac{ len(set(all\_words\_in\_doc)) }{ len(doc) }$$

Find the set of all words observed in the document, and divide it by the number of total words in the document. Let's use this to evalute the diversity of each category in the 20NG dataset.

##### Compute the diversity of each category in the 20NG dataset using the above heuristic

In [32]:
# create a list to store the scores
lexical_diversity_score = []

# loop thru fetch_20newsgroups to clean data 
for category in target_names:
    # extract data from fetch_20newsgroups
    altatheism = fetch_20newsgroups(subset='all',categories= [category])
    # slice data 
    raw_altatheism = altatheism.data
    # clean the data 
    for repl in replace:
        raw_altatheism = [re.sub(repl[0], repl[1], text) for text in raw_altatheism]
    # clean the data with pipeline 
    docs_altatheism = [pipeline(doc) for doc in raw_altatheism[:]]

    # loop through cleaned data to count words
    for i in range(len(docs_altatheism)):
        wordset = set()
        for word in docs_altatheism[i]:
            wordset.add(word)  
    
    # scoring lexical diversity
    altatheism_lex_diver = len(wordset)/len(docs_altatheism)
    
    # put the results into the list
    lexical_diversity_score.append(altatheism_lex_diver)


# create a dic to match the category and it score 
diversity_of_each_category = {}
for i in range(len(target_names)):
    diversity_of_each_category[target_names[i]] = lexical_diversity_score[i]

In [34]:
diversity_of_each_category

{'alt.atheism': 0.03629536921151439,
 'comp.graphics': 0.024665981500513873,
 'comp.os.ms-windows.misc': 0.030456852791878174,
 'comp.sys.ibm.pc.hardware': 0.029531568228105907,
 'comp.sys.mac.hardware': 0.028037383177570093,
 'comp.windows.x': 0.029352226720647773,
 'misc.forsale': 0.031794871794871796,
 'rec.autos': 0.025252525252525252,
 'rec.motorcycles': 0.0321285140562249,
 'rec.sport.baseball': 0.02716297786720322,
 'rec.sport.hockey': 0.028028028028028028,
 'sci.crypt': 0.030272452068617558,
 'sci.electronics': 0.029471544715447155,
 'sci.med': 0.029292929292929294,
 'sci.space': 0.028368794326241134,
 'soc.religion.christian': 0.029087261785356068,
 'talk.politics.guns': 0.03406593406593406,
 'talk.politics.mideast': 0.030851063829787233,
 'talk.politics.misc': 0.03741935483870968,
 'talk.religion.misc': 0.04617834394904458}

##### Explain these scores

1. Is this result real or an artifact of some underlying problem with our data?

According to the equation, the bigger the score is, the more diverse the language is. We could tell that the category of 'talk.religion.misc' has the highest score, which means the highest language diversity, and the category of 'comp.graphics' has the lowest diversity. But this is only the result if we do not consider the length of different kinds of texts. Since it is often necessary for the writer or speaker to re-use several function. This equation is only convincing in a comparison of texts of the same length. So, the result is unreal.

2. What might you do to better evaluate lexical diversity on this data using this scoring function?

It is better to constrain the length of the text because as the text grows, the denominator will grow faster than the numerator, so there is no comparability of linguistic complexity between texts of different lengths.

3. Is this heuristic a good metric for lexical diversity in general?

The results of the data can be used as a reference, but it is not a good calculation method due to the lack of comparability between the results because of it lacks the consideration for the length of the text.

##### Task II

##### Entropy
Entropy is another, perhaps more principled, way by which we can evaluate how diverse, or varied, is a piece of text. Recall the definition of Entropy, $H(P(x))$:

$$ H(P(x)) = \sum_{i=1}^{N} -P(x_{i}) \log P(x_{i}) $$

In the Bag-of-Words (BOW) feature representation of a document, each document is represented by a word count vector, ${x}_{i} \in \mathbb{R}^{N}$ where $N$ is the cardinality of the set of words in the document.

In [35]:
import math
# create a list to store the scores
Entropy_score = []

# loop thru fetch_20newsgroups to clean data 
for category in target_names:
    # extract data from fetch_20newsgroups
    altatheism = fetch_20newsgroups(subset='all',categories= [category])
    # slice data 
    raw_altatheism = altatheism.data
    # clean the data 
    for repl in replace:
        raw_altatheism = [re.sub(repl[0], repl[1], text) for text in raw_altatheism]
    # clean the data with pipeline 
    docs_altatheism = [pipeline(doc) for doc in raw_altatheism[:]]

    # loop through cleaned data to count words
    for i in range(len(docs_altatheism)):
        wordset = set()
        for word in docs_altatheism[i]:
            wordset.add(word)  
    
        # scoring Entropy
        H = 0
        for word in wordset:
            P = docs_altatheism[i].count(word)/len(docs_altatheism[i])
            H -= P * math.log(P)
    
    
            # put the results into the list
            Entropy_score.append(H)


# create a dic to match the category and it score 
Entropy_of_each_category = {}
for i in range(len(target_names)):
    Entropy_of_each_category[target_names[i]] = Entropy_score[i]

In [36]:
Entropy_of_each_category 

{'alt.atheism': 0.1614146009418313,
 'comp.graphics': 0.2419688886211485,
 'comp.os.ms-windows.misc': 0.3925521994579511,
 'comp.sys.ibm.pc.hardware': 0.3982569185221882,
 'comp.sys.mac.hardware': 0.504453623707515,
 'comp.windows.x': 0.5826940838160779,
 'misc.forsale': 0.7268127412537155,
 'rec.autos': 0.9372235328971437,
 'rec.motorcycles': 1.1103329891222051,
 'rec.sport.baseball': 1.1908872768015224,
 'rec.sport.hockey': 1.284715447745436,
 'sci.crypt': 1.3629559078539988,
 'sci.electronics': 1.3686606269182358,
 'sci.med': 1.3976965635331855,
 'sci.space': 1.599754855077846,
 'soc.religion.christian': 1.7784610357247512,
 'talk.politics.guns': 1.8230625606576314,
 'talk.politics.mideast': 1.9814422378641725,
 'talk.politics.misc': 1.9959197255640804,
 'talk.religion.misc': 2.014302459865445}

### Explain this result

1. What does it mean for a distribution to have high or low entropy?

Entropy describes the degree of complexity (uncertainty) of the information. Higher entropy means the more chaotic in the information; lower entropy means there is more certain and homogeneous information.

2. Do these scores make intuitive sense? Any more or less so than the heuristic from Task I?

We can get the degree of complexity of textual information (lexical diversity) by deriving the entropy of different texts, without being limited by the length of the text. It is more heuristic from Task I.

3. Is entropy a good metric for evaluating lexical diversity in general?

Yes. Entropy overcomes the disadvantage of text length and visualizes the complexity of text data.

#### Part II: Document Similarity

##### Task III

Using the definition below, compute the KL-divergence, $K_{DL}$, between the word distributions in each category. This will result in a $K \times K$ matrix of divergence values.

$$ D_{KL}(P||Q) = \sum_{i=1}^{N} P(x_{i}) \log \frac{P(x_{i})}{Q(x_{i})} $$

In [39]:
import pandas as pd
import math

#Take 'alt.atheism' category as an example（considering the large number of computations）

altatheism = fetch_20newsgroups(subset='all',categories= ['alt.atheism'])
# slice data 
raw_altatheism = altatheism.data
# clean the data 
for repl in replace:
    raw_altatheism = [re.sub(repl[0], repl[1], text) for text in raw_altatheism]
    # clean the data with pipeline 
    docs_altatheism = [pipeline(doc) for doc in raw_altatheism[:]]

# avoid the case that Q(Xi)=0
e=0.01
# create a df to store the data
res=pd.DataFrame(columns=list(range(0,len(docs_altatheism))))

# loop through cleaned data to count words
for i in range(len(docs_altatheism)):
    wordset = set()
    for word in docs_altatheism[i]:
        wordset.add(word)  
    for j in range(len(docs_altatheism)):
        # scoring Entropy
        H = 0
        for word in wordset:
            P = docs_altatheism[i].count(word)/len(docs_altatheism[i])
            Q = docs_altatheism[j].count(word)/len(docs_altatheism[j])
            if Q ==0:
                Q = e
                
            dist = P/Q
                
            H += P * math.log(dist)
        res.loc[i,j] = H
print(res)   

          0         1         2         3         4         5         6    \
0         0.0  0.042627  0.032841  0.032102  0.029769  0.040667  0.067757   
1    0.035989       0.0  0.024523  0.027795  0.038158  0.049697  0.060386   
2    0.060906  0.089119       0.0  0.042557  0.051401  0.055726  0.054947   
3    0.031242  0.035837  0.005144       0.0  0.012989  0.033885   0.03535   
4    0.033983  0.051669  0.027388   0.02052       0.0  0.044482  0.033261   
..        ...       ...       ...       ...       ...       ...       ...   
794  0.038377  0.046835  0.028584  0.026457  0.016445  0.041109   0.03833   
795  0.024085  0.047909  0.017936  0.041388  0.023261  0.041234  0.069752   
796  0.024601  0.043674  0.020194   0.02221  0.009854  0.054583  0.035809   
797  0.056529  0.032704  0.019876  0.019766  0.027131  0.050934  0.059921   
798  0.045241  0.052996  0.038643  0.029985  0.026807   0.06011  0.051354   

          7         8         9    ...       789       790       791  \
0  

In [ ]:
## To get the 𝐾×𝐾  matrix of divergence values in each category. 

# create a list to store the scores
KL_score = []

# loop thru fetch_20newsgroups to clean data 
for category in target_names:
    # extract data from fetch_20newsgroups
    altatheism = fetch_20newsgroups(subset='all',categories= [category])
    # slice data 
    raw_altatheism = altatheism.data
    # clean the data 
    for repl in replace:
        raw_altatheism = [re.sub(repl[0], repl[1], text) for text in raw_altatheism]
    # clean the data with pipeline 
    docs_altatheism = [pipeline(doc) for doc in raw_altatheism[:]]

    # avoid the case that Q(Xi)=0
    e=0.01
    # create a df to store the data
    res=pd.DataFrame(columns=list(range(0,len(docs_altatheism))))

    # loop through cleaned data to count words
    for i in range(len(docs_altatheism)):
        wordset = set()
        for word in docs_altatheism[i]:
            wordset.add(word)  
        for j in range(len(docs_altatheism)):

            # scoring Entropy
            H = 0
            for word in wordset:
                P = docs_altatheism[i].count(word)/len(docs_altatheism[i])
                Q = docs_altatheism[j].count(word)/len(docs_altatheism[j])
                if Q ==0:
                    Q = e
                dist = P/Q
                H += P * math.log(dist)
            res.loc[i,j] = H
    
    
            # put the results into the list
            KL_score.append(res)


# create a dic to match the category and it score 
KL_of_each_category = {}
for i in range(len(target_names)):
    KL_of_each_category[target_names[i]] = KL_score[i]

### Explain this result

1. How did you handle any differences in the support for P and Q? What about when Q(x) = 0?

I set Q to a very small number, 0.01. Because Q represents the probability and since Q is in the denominator, it can not be 0. Another way to deal with this is to apply Bayesian smoothing by adding 1 to each count before normalizing the data to change from counts into probabilities.

2. What does it mean for two distributions to have high or low divergence?

KL-divergence, also called relative entropy, is a type of statistical distance: a measure of how one probability distribution P is different from a second, reference probability distribution Q. When two probability distributions are identical, their relative entropy is 0. When their differences increase, the relative entropy increases. So, a high divergence means the probability distribution of two words is very different, vice versa.  

3. Do these similarity scores make sense intuitively?

It is not symmetric in the distributions of P and Q, so these similarity scores does not make sense intuitively.

4. Is the resultant $K \times K$ matrix symmetric? Why is this the case?

 $K \times K$ matrix is not symmetric. The KL from p(x) to q(x) is generally not the same as the KL from q(x) to p(x).It is because the Kullback-Leibler divergence is not symmetric. Roughly speaking, it's because the two arguments of the KL divergence as different kinds of things: the first argument is empirical data, and the second argument is a model we compare the data to.

5. Is $D_{KL}$ a good measure of the similarity in this context? 

$D_{KL}$ a good measure of the similarity in this context. And compared to entropy, relative entropy is better because it uses the current best information to estimate the difference in average surprise. While maximizing absolute entropy differences will give the biggest change in expected surprise, maximizing relative entropy differences will do a better job at giving the biggest difference in *actual* surprise. Relative entropy, in other words, allows us to correct for previous bad estimates of average surprise, and substitute in the best estimate we currently have.